# How should we set the price of a product to get the maximum revenue from it?

Price optimization is essential for companies as it maximizes revenue and profit by ensuring products are neither underpriced nor overpriced. It provides **a competitive edge, positions products strategically, and allows dynamic responses to market changes**. 

Optimized pricing offers **insights into customer behavior, aids in market segmentation, and enhances inventory management** by balancing supply and demand.

It fosters long-term growth and brand loyalty through fair pricing and ensures financial health for investments in innovation.

![Price Optimization](images/price-optimization-opportunities.svg)

## 🏨 Background

You are supporting a cafe with a project aimed to increase revenue from their items. They believe that they can use data science to help them!

They have asked you to **develop a pricing strategy to maximize their sales and revenues for their products**. They intend to use the results of your work to find their best pricing strategy.

## 💾 The Data

They have provided you with their transactions data in a file called `cafe_transaction_store.csv`, which contains the following:

| Column     | Description              |
|------------|--------------------------|
| `calendar_date` | Date of the transaction.|
| `holiday` | Whether the transaction date was on specific holiday. |
| `price` | Unit price of the item. |
| `quantity` | Sales quantity of the item. |
| `item_name` | The type of item was sold. |

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import optuna
from plotly.subplots import make_subplots
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from pycaret.regression import setup, compare_models, create_model, evaluate_model

In [2]:
df = pd.read_csv('cafe_transaction_store.csv')
df

,calendar_date,holiday,price,quantity,item_name
0,2012-01-01,New Year,155.0,920,BURGER
1,2012-01-01,New Year,127.3,440,COFFEE
2,2012-01-01,New Year,127.5,360,LEMONADE
3,2012-01-01,New Year,126.0,600,SANDWICH
4,2012-01-02,New Year,155.0,1400,BURGER
...,...,...,...,...,...
5415,2015-09-09,NaN,112.6,1080,SANDWICH
5416,2015-09-10,NaN,145.0,1920,BURGER
5417,2015-09-10,NaN,121.9,600,COFFEE
5418,2015-09-10,NaN,115.0,360,LEMONADE


![project_pipeline](images/pipeline.svg)

## 1. Exploratory Data Analysis (EDA)

Our dataset has **some missing values**. Let's take a more detailed look!

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5420 entries, 0 to 5419
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   calendar_date  5420 non-null   object 
 1   holiday        420 non-null    object 
 2   price          5420 non-null   float64
 3   quantity       5420 non-null   int64  
 4   item_name      5420 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 211.8+ KB


>**Q:** Why do we have so many missing values on holiday column?

In [4]:
df.holiday.value_counts()

holiday
Luner New Year          112
National Day             84
Qing Ming Festival       48
Labor Day                48
Dragon Boat Festivel     48
New Year                 40
Mid-Autumn Day           28
WWII Celebration         12
Name: count, dtype: int64

Hmm... It stores only the holiday information in it. Other dates must be a normal date.

>**Q:** Do we really need all this informations?

In [5]:
pd.concat([pd.get_dummies(df['holiday'], dtype=int), df['quantity']], axis=1).corr()

,Dragon Boat Festivel,Labor Day,Luner New Year,Mid-Autumn Day,National Day,New Year,Qing Ming Festival,WWII Celebration,quantity
Dragon Boat Festivel,1.000000,-0.008935,-0.013731,-0.006812,-0.011860,-0.008151,-0.008935,-0.004453,-0.046793
Labor Day,-0.008935,1.000000,-0.013731,-0.006812,-0.011860,-0.008151,-0.008935,-0.004453,-0.042778
Luner New Year,-0.013731,-0.013731,1.000000,-0.010468,-0.018225,-0.012525,-0.013731,-0.006843,-0.060076
Mid-Autumn Day,-0.006812,-0.006812,-0.010468,1.000000,-0.009041,-0.006214,-0.006812,-0.003395,-0.037270
National Day,-0.011860,-0.011860,-0.018225,-0.009041,1.000000,-0.010819,-0.011860,-0.005910,-0.058567
New Year,-0.008151,-0.008151,-0.012525,-0.006214,-0.010819,1.000000,-0.008151,-0.004062,-0.028344
Qing Ming Festival,-0.008935,-0.008935,-0.013731,-0.006812,-0.011860,-0.008151,1.000000,-0.004453,-0.047102
WWII Celebration,-0.004453,-0.004453,-0.006843,-0.003395,-0.005910,-0.004062,-0.004453,1.000000,-0.021241
quantity,-0.046793,-0.042778,-0.060076,-0.037270,-0.058567,-0.028344,-0.047102,-0.021241,1.000000


When we look into table above, the largest correlation value is **-0.060076** with quantity feature. Therefore we can drop the holiday information in our dataset.

In [6]:
df.drop(columns='holiday', inplace=True)

For analyzing the date effects in our dataset, we must convert it to datetime feature.

In [7]:
df['calendar_date'] = pd.to_datetime(df['calendar_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5420 entries, 0 to 5419
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   calendar_date  5420 non-null   datetime64[ns]
 1   price          5420 non-null   float64       
 2   quantity       5420 non-null   int64         
 3   item_name      5420 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 169.5+ KB


In [8]:
df['day_name'] = df['calendar_date'].dt.day_name()
df['day_number'] = df['calendar_date'].dt.day_of_week
df['month_name'] = df['calendar_date'].dt.month_name()
df['month_number'] = df['calendar_date'].dt.month
df['year'] = df['calendar_date'].dt.year
df

,calendar_date,price,quantity,item_name,day_name,day_number,month_name,month_number,year
0,2012-01-01,155.0,920,BURGER,Sunday,6,January,1,2012
1,2012-01-01,127.3,440,COFFEE,Sunday,6,January,1,2012
2,2012-01-01,127.5,360,LEMONADE,Sunday,6,January,1,2012
3,2012-01-01,126.0,600,SANDWICH,Sunday,6,January,1,2012
4,2012-01-02,155.0,1400,BURGER,Monday,0,January,1,2012
...,...,...,...,...,...,...,...,...,...
5415,2015-09-09,112.6,1080,SANDWICH,Wednesday,2,September,9,2015
5416,2015-09-10,145.0,1920,BURGER,Thursday,3,September,9,2015
5417,2015-09-10,121.9,600,COFFEE,Thursday,3,September,9,2015
5418,2015-09-10,115.0,360,LEMONADE,Thursday,3,September,9,2015


Let's analyze the date effects above with graphs.

In [9]:
quantity_by_day = pd.DataFrame(df.groupby(['day_number','day_name'])['quantity'].mean().reset_index())
quantity_by_day

,day_number,day_name,quantity
0,0,Monday,968.911917
1,1,Tuesday,957.409326
2,2,Wednesday,966.927083
3,3,Thursday,962.746114
4,4,Friday,968.793970
5,5,Saturday,693.072917
6,6,Sunday,685.803109


In [10]:
px.line(quantity_by_day, x='day_name', y='quantity', markers='*')

We can clearly say that there is a **significant difference in quantity of sales between weekdays and weekends**! We must use this information in our model.

In [11]:
df['is_weekend'] = df['day_number'].apply(lambda day: 1 if day >= 5 else 0)
df.drop(columns=['day_name','day_number'], inplace=True)
df

,calendar_date,price,quantity,item_name,month_name,month_number,year,is_weekend
0,2012-01-01,155.0,920,BURGER,January,1,2012,1
1,2012-01-01,127.3,440,COFFEE,January,1,2012,1
2,2012-01-01,127.5,360,LEMONADE,January,1,2012,1
3,2012-01-01,126.0,600,SANDWICH,January,1,2012,1
4,2012-01-02,155.0,1400,BURGER,January,1,2012,0
...,...,...,...,...,...,...,...,...
5415,2015-09-09,112.6,1080,SANDWICH,September,9,2015,0
5416,2015-09-10,145.0,1920,BURGER,September,9,2015,0
5417,2015-09-10,121.9,600,COFFEE,September,9,2015,0
5418,2015-09-10,115.0,360,LEMONADE,September,9,2015,0


In [12]:
quantity_by_month = pd.DataFrame(df.groupby(['month_number','month_name'])['quantity'].mean().reset_index())
quantity_by_month

,month_number,month_name,quantity
0,1,January,942.338710
1,2,February,873.214286
2,3,March,863.129771
3,4,April,822.083333
4,5,May,863.225806
5,6,June,855.083333
6,7,July,886.370968
7,8,August,905.161290
8,9,September,886.200000
9,10,October,859.784946


In [13]:
px.line(quantity_by_month, x='month_name', y='quantity', markers='*')

We cannot say anything on the graph above. Let's look into correlation values.

In [14]:
pd.concat([pd.get_dummies(df['month_name'], dtype=int), df['quantity']], axis=1).corr()

,April,August,December,February,January,July,June,March,May,November,October,September,quantity
April,1.000000,-0.098932,-0.084619,-0.093569,-0.098932,-0.098932,-0.097166,-0.101977,-0.098932,-0.083144,-0.084619,-0.087990,-0.039477
August,-0.098932,1.000000,-0.086158,-0.095270,-0.100731,-0.100731,-0.098932,-0.103831,-0.100731,-0.084656,-0.086158,-0.089590,0.011495
December,-0.084619,-0.086158,1.000000,-0.081486,-0.086158,-0.086158,-0.084619,-0.088809,-0.086158,-0.072408,-0.073693,-0.076628,0.059958
February,-0.093569,-0.095270,-0.081486,1.000000,-0.095270,-0.095270,-0.093569,-0.098202,-0.095270,-0.080066,-0.081486,-0.084733,-0.007928
January,-0.098932,-0.100731,-0.086158,-0.095270,1.000000,-0.100731,-0.098932,-0.103831,-0.100731,-0.084656,-0.086158,-0.089590,0.034626
July,-0.098932,-0.100731,-0.086158,-0.095270,-0.100731,1.000000,-0.098932,-0.103831,-0.100731,-0.084656,-0.086158,-0.089590,-0.000196
June,-0.097166,-0.098932,-0.084619,-0.093569,-0.098932,-0.098932,1.000000,-0.101977,-0.098932,-0.083144,-0.084619,-0.087990,-0.019312
March,-0.101977,-0.103831,-0.088809,-0.098202,-0.103831,-0.103831,-0.101977,1.000000,-0.103831,-0.087261,-0.088809,-0.092347,-0.015107
May,-0.098932,-0.100731,-0.086158,-0.095270,-0.100731,-0.100731,-0.098932,-0.103831,1.000000,-0.084656,-0.086158,-0.089590,-0.014597
November,-0.083144,-0.084656,-0.072408,-0.080066,-0.084656,-0.084656,-0.083144,-0.087261,-0.084656,1.000000,-0.072408,-0.075293,0.011435


In [15]:
df.drop(columns=['month_name','month_number'], inplace=True)
df

,calendar_date,price,quantity,item_name,year,is_weekend
0,2012-01-01,155.0,920,BURGER,2012,1
1,2012-01-01,127.3,440,COFFEE,2012,1
2,2012-01-01,127.5,360,LEMONADE,2012,1
3,2012-01-01,126.0,600,SANDWICH,2012,1
4,2012-01-02,155.0,1400,BURGER,2012,0
...,...,...,...,...,...,...
5415,2015-09-09,112.6,1080,SANDWICH,2015,0
5416,2015-09-10,145.0,1920,BURGER,2015,0
5417,2015-09-10,121.9,600,COFFEE,2015,0
5418,2015-09-10,115.0,360,LEMONADE,2015,0


What about years?

In [16]:
quantity_by_year = pd.DataFrame(df.groupby('year')['quantity'].mean().reset_index())
quantity_by_year

,year,quantity
0,2012,880.684932
1,2013,890.725806
2,2014,882.246575
3,2015,895.810277


In [17]:
px.line(quantity_by_year, x='year', y='quantity', markers='*')

In [18]:
df.drop(columns='year', inplace=True)
df

,calendar_date,price,quantity,item_name,is_weekend
0,2012-01-01,155.0,920,BURGER,1
1,2012-01-01,127.3,440,COFFEE,1
2,2012-01-01,127.5,360,LEMONADE,1
3,2012-01-01,126.0,600,SANDWICH,1
4,2012-01-02,155.0,1400,BURGER,0
...,...,...,...,...,...
5415,2015-09-09,112.6,1080,SANDWICH,0
5416,2015-09-10,145.0,1920,BURGER,0
5417,2015-09-10,121.9,600,COFFEE,0
5418,2015-09-10,115.0,360,LEMONADE,0


We analyzed our whole dataset until now. Let's continue to examine more detail.

In [19]:
sales_by_date = pd.DataFrame(df.groupby(['item_name','calendar_date'])[['price','quantity']].mean().reset_index())
sales_by_date

,item_name,calendar_date,price,quantity
0,BURGER,2012-01-01,155.0,920.0
1,BURGER,2012-01-02,155.0,1400.0
2,BURGER,2012-01-03,155.0,1240.0
3,BURGER,2012-01-04,155.0,1760.0
4,BURGER,2012-01-05,155.0,2080.0
...,...,...,...,...
5387,SANDWICH,2015-09-06,112.6,1080.0
5388,SANDWICH,2015-09-07,112.6,1000.0
5389,SANDWICH,2015-09-08,112.6,1000.0
5390,SANDWICH,2015-09-09,112.6,1080.0


In [20]:
px.line(sales_by_date, x='calendar_date', y='quantity', color='item_name')

>**Q:** What does the graph above say?

In [21]:
sales_by_date['item_quantity_7d_ma'] = sales_by_date.groupby('item_name')['quantity'].transform(lambda x: x.rolling(window=7).mean())
sales_by_date

,item_name,calendar_date,price,quantity,item_quantity_7d_ma
0,BURGER,2012-01-01,155.0,920.0,NaN
1,BURGER,2012-01-02,155.0,1400.0,NaN
2,BURGER,2012-01-03,155.0,1240.0,NaN
3,BURGER,2012-01-04,155.0,1760.0,NaN
4,BURGER,2012-01-05,155.0,2080.0,NaN
...,...,...,...,...,...
5387,SANDWICH,2015-09-06,112.6,1080.0,925.714286
5388,SANDWICH,2015-09-07,112.6,1000.0,914.285714
5389,SANDWICH,2015-09-08,112.6,1000.0,897.142857
5390,SANDWICH,2015-09-09,112.6,1080.0,902.857143


In [22]:
px.line(sales_by_date, x='calendar_date', y='item_quantity_7d_ma', color='item_name')

It follows **quite similar quantity of sales** for each items of our cafe in 7 days moving average time periods.

Let's examine price-quantity relationship on our different items.

In [23]:
px.scatter(df, x='price', y='quantity', color='item_name', trendline='ols')

We can use box plot visualization for detecting outliers observations in our dataset.

In [24]:
px.box(df, x='price', color='item_name')

In [25]:
coffee_outliers = df[(df['item_name'] == 'COFFEE') & (df['price'] > 132.7)].index
burger_outliers = df[(df['item_name'] == 'BURGER') & (df['price'] < 140)].index

df.drop(coffee_outliers, inplace=True)
df.drop(burger_outliers, inplace=True)
df

,calendar_date,price,quantity,item_name,is_weekend
0,2012-01-01,155.0,920,BURGER,1
1,2012-01-01,127.3,440,COFFEE,1
2,2012-01-01,127.5,360,LEMONADE,1
3,2012-01-01,126.0,600,SANDWICH,1
4,2012-01-02,155.0,1400,BURGER,0
...,...,...,...,...,...
5415,2015-09-09,112.6,1080,SANDWICH,0
5416,2015-09-10,145.0,1920,BURGER,0
5417,2015-09-10,121.9,600,COFFEE,0
5418,2015-09-10,115.0,360,LEMONADE,0


In [26]:
df.drop(columns='calendar_date', inplace=True)
df

,price,quantity,item_name,is_weekend
0,155.0,920,BURGER,1
1,127.3,440,COFFEE,1
2,127.5,360,LEMONADE,1
3,126.0,600,SANDWICH,1
4,155.0,1400,BURGER,0
...,...,...,...,...
5415,112.6,1080,SANDWICH,0
5416,145.0,1920,BURGER,0
5417,121.9,600,COFFEE,0
5418,115.0,360,LEMONADE,0


## 2. Modeling

For implementing the modeling step more easily, we will use `pycaret.regression` package.

In [27]:
setup = setup(df, target='quantity', train_size=0.8, fold=5)

,Description,Value
0,Session id,7129
1,Target,quantity
2,Target type,Regression
3,Original data shape,"(5412, 4)"
4,Transformed data shape,"(5412, 7)"
5,Transformed train set shape,"(4329, 7)"
6,Transformed test set shape,"(1083, 7)"
7,Numeric features,2
8,Categorical features,1
9,Preprocess,True


In [28]:
compare_models(sort='R2')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,76.8191,13648.1450,116.3948,0.9476,0.1400,0.1057,0.0080
xgboost,Extreme Gradient Boosting,76.8190,13648.1383,116.3948,0.9476,0.1400,0.1057,0.0200
et,Extra Trees Regressor,76.8191,13648.1450,116.3948,0.9476,0.1400,0.1057,0.0260
rf,Random Forest Regressor,76.9618,13662.3609,116.4599,0.9476,0.1402,0.1059,0.0300
gbr,Gradient Boosting Regressor,76.9869,13744.8166,116.8495,0.9472,0.1409,0.1061,0.0240
lightgbm,Light Gradient Boosting Machine,77.2678,13800.3466,117.0732,0.9470,0.1417,0.1065,0.3560
knn,K Neighbors Regressor,85.2983,15837.3520,125.3643,0.9392,0.1492,0.1143,0.0100
ada,AdaBoost Regressor,103.8142,20276.5894,142.1699,0.9221,0.1880,0.1482,0.0140
lar,Least Angle Regression,102.7812,21044.2461,144.8337,0.9192,0.2211,0.1453,0.0080
br,Bayesian Ridge,102.7815,21044.2495,144.8336,0.9192,0.2210,0.1453,0.0100


DecisionTreeRegressor(random_state=7129)

In [29]:
xgb = create_model('xgboost', return_train_score=True)

In [30]:
evaluate_model(xgb)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

It seems that **XGBoost algorithm and its model performance is well**. We can use it to obtain our model.

Now let's tune our model to perform slightly better.

In [31]:
ohe = OneHotEncoder(sparse_output=False)

train_encoded = ohe.fit_transform(setup.train['item_name'].values.reshape(-1,1))

encoded_train_df = pd.DataFrame(train_encoded, columns=ohe.get_feature_names_out(['item_name']))
df_train = pd.concat([setup.train.drop(columns=['item_name']).reset_index(drop=True), encoded_train_df], axis=1)
df_train

,price,is_weekend,quantity,item_name_BURGER,item_name_COFFEE,item_name_LEMONADE,item_name_SANDWICH
0,127.500000,0,320,0.0,0.0,1.0,0.0
1,145.000000,0,1880,1.0,0.0,0.0,0.0
2,155.000000,0,1680,1.0,0.0,0.0,0.0
3,109.699997,0,920,0.0,1.0,0.0,0.0
4,114.800003,0,1000,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
4324,132.699997,0,720,0.0,1.0,0.0,0.0
4325,132.699997,0,640,0.0,1.0,0.0,0.0
4326,104.500000,1,920,0.0,0.0,0.0,1.0
4327,126.400002,0,320,0.0,0.0,1.0,0.0


In [32]:
test_encoded = ohe.transform(setup.test['item_name'].values.reshape(-1,1))

encoded_test_df = pd.DataFrame(test_encoded, columns=ohe.get_feature_names_out(['item_name']))
df_test = pd.concat([setup.test.drop(columns=['item_name']).reset_index(drop=True), encoded_test_df], axis=1)
df_test

,price,is_weekend,quantity,item_name_BURGER,item_name_COFFEE,item_name_LEMONADE,item_name_SANDWICH
0,145.000000,1,1280,1.0,0.0,0.0,0.0
1,155.000000,1,1200,1.0,0.0,0.0,0.0
2,126.599998,0,560,0.0,1.0,0.0,0.0
3,155.000000,1,1400,1.0,0.0,0.0,0.0
4,123.500000,0,640,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1078,126.000000,0,880,0.0,0.0,0.0,1.0
1079,126.400002,0,400,0.0,0.0,1.0,0.0
1080,124.199997,1,720,0.0,0.0,0.0,1.0
1081,132.699997,1,480,0.0,1.0,0.0,0.0


In [33]:
X_train = df_train.drop(columns='quantity')
X_test = df_test.drop(columns='quantity')
y_train = df_train['quantity']
y_test = df_test['quantity']

def maximize_r2(trial):
    
    lr_range = trial.suggest_float('learning_rate', 0.01, 0.5)
    ne_range = trial.suggest_int('n_estimators', 100, 500)
    md_range = trial.suggest_int('max_depth', 2, 10)
    ss_range = trial.suggest_float('subsample', 0.5, 1)
    cs_range = trial.suggest_float('colsample_bytree', 0.5, 1)
    gm_range = trial.suggest_float('gamma', 0, 0.5)
    
    regressor = XGBRegressor(learning_rate=lr_range, n_estimators=ne_range, max_depth=md_range,
                             subsample=ss_range, colsample_bytree=cs_range, gamma=gm_range)

    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    error = r2_score(y_test, y_pred)

    return error

study = optuna.create_study(study_name='Maximizing R2', direction='maximize')
study.optimize(maximize_r2, n_trials=100)

[I 2024-09-02 17:47:16,515] A new study created in memory with name: Maximizing R2
[I 2024-09-02 17:47:16,678] Trial 0 finished with value: 0.9321779350750143 and parameters: {'learning_rate': 0.35170140670887534, 'n_estimators': 285, 'max_depth': 2, 'subsample': 0.8960359694407802, 'colsample_bytree': 0.8292292054686012, 'gamma': 0.0785061708425176}. Best is trial 0 with value: 0.9321779350750143.
[I 2024-09-02 17:47:17,233] Trial 1 finished with value: 0.932194399101819 and parameters: {'learning_rate': 0.45781611882058704, 'n_estimators': 339, 'max_depth': 9, 'subsample': 0.8086110302096404, 'colsample_bytree': 0.9959861519218622, 'gamma': 0.233797972951522}. Best is trial 1 with value: 0.932194399101819.
[I 2024-09-02 17:47:17,593] Trial 2 finished with value: 0.9323359609082568 and parameters: {'learning_rate': 0.29896591435877845, 'n_estimators': 219, 'max_depth': 10, 'subsample': 0.5462575544442603, 'colsample_bytree': 0.9768596737072981, 'gamma': 0.12797903185002157}. Best is t

In [34]:
study.best_trial

FrozenTrial(number=83, state=1, values=[0.9325888925449033], datetime_start=datetime.datetime(2024, 9, 2, 17, 47, 45, 312016), datetime_complete=datetime.datetime(2024, 9, 2, 17, 47, 45, 690412), params={'learning_rate': 0.016265789134438696, 'n_estimators': 458, 'max_depth': 7, 'subsample': 0.9493879785488181, 'colsample_bytree': 0.5009384333338421, 'gamma': 0.19264120446400324}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.5, log=False, low=0.01, step=None), 'n_estimators': IntDistribution(high=500, log=False, low=100, step=1), 'max_depth': IntDistribution(high=10, log=False, low=2, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=0.5, log=False, low=0.0, step=None)}, trial_id=83, value=None)

In [35]:
study.best_params

{'learning_rate': 0.016265789134438696,
 'n_estimators': 458,
 'max_depth': 7,
 'subsample': 0.9493879785488181,
 'colsample_bytree': 0.5009384333338421,
 'gamma': 0.19264120446400324}

In [36]:
best_xgb = XGBRegressor(learning_rate=0.18255008856295465,
                        n_estimators=252,
                        max_depth=3,
                        subsample=0.8666115132971169,
                        colsample_bytree=0.9430082838305646,
                        gamma=0.32916036280407013)

best_xgb.fit(X_train, y_train)
preds = best_xgb.predict(X_test)
r2_score(y_test, preds)

0.9322588096193645

## 3. Optimization

Now, let's predict the quantity of sales and its revenue by using our final model.

In [37]:
df_test['quantity_prediction'] = preds
df_test

,price,is_weekend,quantity,item_name_BURGER,item_name_COFFEE,item_name_LEMONADE,item_name_SANDWICH,quantity_prediction
0,145.000000,1,1280,1.0,0.0,0.0,0.0,1312.717529
1,155.000000,1,1200,1.0,0.0,0.0,0.0,1230.665649
2,126.599998,0,560,0.0,1.0,0.0,0.0,592.792236
3,155.000000,1,1400,1.0,0.0,0.0,0.0,1230.665649
4,123.500000,0,640,0.0,1.0,0.0,0.0,631.476257
...,...,...,...,...,...,...,...,...
1078,126.000000,0,880,0.0,0.0,0.0,1.0,930.100708
1079,126.400002,0,400,0.0,0.0,1.0,0.0,401.312225
1080,124.199997,1,720,0.0,0.0,0.0,1.0,686.738647
1081,132.699997,1,480,0.0,1.0,0.0,0.0,462.609100


In [38]:
df_test['revenue_prediction'] = df_test['price'] * df_test['quantity_prediction']
df_test

,price,is_weekend,quantity,item_name_BURGER,item_name_COFFEE,item_name_LEMONADE,item_name_SANDWICH,quantity_prediction,revenue_prediction
0,145.000000,1,1280,1.0,0.0,0.0,0.0,1312.717529,190344.046875
1,155.000000,1,1200,1.0,0.0,0.0,0.0,1230.665649,190753.171875
2,126.599998,0,560,0.0,1.0,0.0,0.0,592.792236,75047.500000
3,155.000000,1,1400,1.0,0.0,0.0,0.0,1230.665649,190753.171875
4,123.500000,0,640,0.0,1.0,0.0,0.0,631.476257,77987.320312
...,...,...,...,...,...,...,...,...,...
1078,126.000000,0,880,0.0,0.0,0.0,1.0,930.100708,117192.687500
1079,126.400002,0,400,0.0,0.0,1.0,0.0,401.312225,50725.867188
1080,124.199997,1,720,0.0,0.0,0.0,1.0,686.738647,85292.937500
1081,132.699997,1,480,0.0,1.0,0.0,0.0,462.609100,61388.226562


Let's visualize our findings.

In [39]:
one_hot_columns = ['item_name_BURGER', 'item_name_COFFEE', 'item_name_LEMONADE', 'item_name_SANDWICH']

df_test['item_name'] = df_test[one_hot_columns].idxmax(axis=1).str.replace('item_name_', '')
df_test.drop(columns=one_hot_columns, inplace=True)
df_test

,price,is_weekend,quantity,quantity_prediction,revenue_prediction,item_name
0,145.000000,1,1280,1312.717529,190344.046875,BURGER
1,155.000000,1,1200,1230.665649,190753.171875,BURGER
2,126.599998,0,560,592.792236,75047.500000,COFFEE
3,155.000000,1,1400,1230.665649,190753.171875,BURGER
4,123.500000,0,640,631.476257,77987.320312,COFFEE
...,...,...,...,...,...,...
1078,126.000000,0,880,930.100708,117192.687500,SANDWICH
1079,126.400002,0,400,401.312225,50725.867188,LEMONADE
1080,124.199997,1,720,686.738647,85292.937500,SANDWICH
1081,132.699997,1,480,462.609100,61388.226562,COFFEE


In [40]:
item_max_revenue = df_test.groupby(['item_name','is_weekend'])['revenue_prediction'].max().reset_index()
item_max_revenue

,item_name,is_weekend,revenue_prediction
0,BURGER,0,284015.968750
1,BURGER,1,191981.484375
2,COFFEE,0,79796.640625
3,COFFEE,1,61388.226562
4,LEMONADE,0,105934.453125
5,LEMONADE,1,42447.421875
6,SANDWICH,0,126837.320312
7,SANDWICH,1,90764.875000


In [41]:
fig = make_subplots(rows=4, cols=2, subplot_titles=['BURGER Weekday', 'BURGER Weekend', 'COFFEE Weekday', 'COFFEE Weekend', 
                                                    'LEMONADE Weekday', 'LEMONADE Weekend', 'SANDWICH Weekday', 'SANDWICH Weekend'])

row_no = 1
col_no = 1

for item, color in zip(['BURGER', 'COFFEE', 'LEMONADE', 'SANDWICH'], ['red', 'black', '#CCCC00', 'green']):
    
    for is_weekend in [0, 1]:
        subplot_df = df_test[(df_test['item_name'] == item) & (df_test['is_weekend'] == is_weekend)]
        label = 'Weekend' if is_weekend == 1 else 'Weekday'
        
        fig.add_trace(go.Scatter(x=subplot_df['price'], y=subplot_df['revenue_prediction'], name=f'{item} {label}',
                                 mode='markers', marker=dict(color=color)), row=row_no, col=col_no)
        
        fig.add_shape(
            type='line',
            x0=subplot_df['price'].min(),
            x1=subplot_df['price'].max(),
            y0=subplot_df['revenue_prediction'].max(),
            y1=subplot_df['revenue_prediction'].max(),
            line=dict(color=color, width=2, dash='dash'), row=row_no, col=col_no)
        
        fig.update_xaxes(title_text='Price', row=row_no, col=col_no)
        fig.update_yaxes(title_text='Revenue', row=row_no, col=col_no)

        if col_no == 1:
            col_no += 1
        else:
            col_no = 1
            row_no += 1

fig.update_layout(title_text='Price vs Revenue Relationships by Item & Time', title_x=0.5, height=1600)
fig.show()

>**Q:** Is our findings above enough? What if we will sell our burger lower than 140$ or lemonade greater than 134$?

In [42]:
def objective(trial, is_weekend, item_name):

    price = trial.suggest_float('price', 50, 250)

    pred_encoded = ohe.transform(pd.Series(item_name).values.reshape(-1,1))
    encoded_pred_df = pd.DataFrame(pred_encoded, columns=ohe.get_feature_names_out(['item_name']))
    pred_df = pd.DataFrame({'price': price, 'is_weekend': is_weekend}, index=[0])
    df_pred = pd.concat([pred_df.reset_index(drop=True), encoded_pred_df], axis=1)

    predicted_quantity = best_xgb.predict(df_pred)

    predicted_revenue = price * predicted_quantity

    return predicted_revenue

def maximize_revenue(is_weekend, item_name):
    
    study = optuna.create_study(study_name='Maximizing Revenue', direction='maximize')
    study.optimize(lambda trial: objective(trial, is_weekend, item_name), n_trials=100)

    best_price = study.best_params['price']
    best_revenue = study.best_value

    print(f'{item_name} Best Price: {best_price}')
    print(f'Maximum Revenue: {best_revenue}')

In [43]:
maximize_revenue(is_weekend=1, item_name='BURGER')

[I 2024-09-02 17:47:52,078] A new study created in memory with name: Maximizing Revenue
[I 2024-09-02 17:47:52,082] Trial 0 finished with value: 223565.40625 and parameters: {'price': 201.9318149684711}. Best is trial 0 with value: 223565.40625.
[I 2024-09-02 17:47:52,086] Trial 1 finished with value: 56452.8671875 and parameters: {'price': 89.03463956511443}. Best is trial 0 with value: 223565.40625.
[I 2024-09-02 17:47:52,089] Trial 2 finished with value: 32418.404296875 and parameters: {'price': 51.12868375042028}. Best is trial 0 with value: 223565.40625.
[I 2024-09-02 17:47:52,091] Trial 3 finished with value: 74502.875 and parameters: {'price': 135.68803127641408}. Best is trial 0 with value: 223565.40625.
[I 2024-09-02 17:47:52,093] Trial 4 finished with value: 237960.734375 and parameters: {'price': 214.9341535806478}. Best is trial 4 with value: 237960.734375.
[I 2024-09-02 17:47:52,095] Trial 5 finished with value: 194751.625 and parameters: {'price': 148.3576050809718}. Best

BURGER Best Price: 249.96334637453808
Maximum Revenue: 276742.71875
